In [1]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf  # For tf.data
import matplotlib.pyplot as plt
import keras
from keras import layers
from keras.applications import EfficientNetB0
import re
import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

2025-05-20 08:49:32.198183: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747730972.437161      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747730972.504478      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
imgs = glob.glob(r'/kaggle/input/mushrooms-classification-common-genuss-images/Mushrooms/*/*')
labels = [el.split('/')[5] for el in imgs ]
SIZE = 224 

In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
X_train, X_test, y_train, y_test = train_test_split(imgs, labels, test_size=0.3, random_state=1, stratify=labels)
train = pd.DataFrame(list(zip(X_train,y_train)), columns =['X','y'])
test = pd.DataFrame(list(zip(X_test,y_test)), columns =['X','y'])

datagen = ImageDataGenerator(
        rescale=1./255)
valid_gen = ImageDataGenerator(rescale=1./255)



train_gen = datagen.flow_from_dataframe(train, x_col='X', y_col="y", target_size=(SIZE, SIZE), interpolation='lanczos',)
test_gen = valid_gen.flow_from_dataframe(test, x_col='X', y_col="y", target_size=(SIZE, SIZE), interpolation='lanczos', )

Found 4699 validated image filenames belonging to 9 classes.
Found 2015 validated image filenames belonging to 9 classes.


In [5]:
efb0 = EfficientNetB0()
len(efb0.layers)

2025-05-20 08:51:54.015930: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


21834768/21834768 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


241

In [9]:


m1  = keras.Sequential()

#m1.add()
for i, l in enumerate(efb0.layers):
    m1.add(efb0.layers[i])
    if i==7:
        break



for i, l in enumerate(efb0.layers):
    if re.match('^block1|^block2',l.name) != None:
        m1.add(efb0.layers[i]) 

In [10]:
m1.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ normalization (Normalization)        │ (None, 224, 224, 3)         │               7 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rescaling_1 (Rescaling)              │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ stem_conv_pad (ZeroPadding2D)        │ (None, 225, 225, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ stem_conv (Conv2D)                   │ (None, 112, 112, 32)        │             864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ stem_bn (BatchNormalization)         │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ stem_activation (Activation)         │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1a_dwconv (DepthwiseConv2D)     │ (None, 112, 112, 32)        │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1a_dwconv (DepthwiseConv2D)     │ (None, 112, 112, 32)        │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1a_bn (BatchNormalization)      │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1a_activation (Activation)      │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1a_se_squeeze                   │ (None, 32)                  │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1a_se_reshape (Reshape)         │ (None, 1, 1, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1a_se_reduce (Conv2D)           │ (None, 1, 1, 8)             │             264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1a_se_expand (Conv2D)           │ (None, 1, 1, 32)            │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1a_se_excite (Multiply)         │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1a_project_conv (Conv2D)        │ (None, 112, 112, 16)        │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1a_project_bn                   │ (None, 112, 112, 16)        │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2a_expand_conv (Conv2D)         │ (None, 112, 112, 96)        │           1,536 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2a_expand_bn                    │ (None, 112, 112, 96)        │             384 │
│ (BatchNormalization)                 │                             │              

 Total params: 20,313 (79.35 KB)

 Trainable params: 19,090 (74.57 KB)

 Non-trainable params: 1,223 (4.78 KB)

In [ ]:
def build_model(model, num_classes=9, shape=(224,224,3)):
    inputs = tf.keras.layers.Input(shape=shape)
    
    model.trainable = True

    # Rebuild top
    x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Dropout(0.5, name="top_dropout")(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = keras.Model(inputs, outputs, name="EfficientNetB1")
    
    return model

In [19]:
inputs = tf.keras.layers.Input(shape=(224,224, 3))
output = m1.layers[-1].output 
x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(output)
x = tf.keras.layers.BatchNormalization()(x)
    
x = tf.keras.layers.Dropout(0.5, name="top_dropout")(x)
outputs = tf.keras.layers.Dense(9, activation="softmax", name="pred")(x)

    # Compile
m1 = keras.Model(inputs, outputs, name="EfficientNetB1")


In [11]:
x = m1.layers[-1].output 
x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5, name="top_dropout")(x)
outputs = tf.keras.layers.Dense(9, activation="softmax", name="pred")(x)



#predictions = layers.Dense(10, activation = "softmax")(x)
m1 = keras.Model(inputs = m1.input, outputs = outputs)

AttributeError: The layer sequential_3 has never been called and thus has no defined input.

In [20]:
### TRy with MODEL same shit
m1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



history = m1.fit(
    train_gen,
    epochs=15,
    batch_size=64
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15


KeyError: 'Exception encountered when calling Functional.call().\n\n\x1b[1m133962997043728\x1b[0m\n\nArguments received by Functional.call():\n  • inputs=tf.Tensor(shape=(None, 224, 224, 3), dtype=float32)\n  • training=True\n  • mask=None'